In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_1 = np.load("burgers_50k-1.npy", mmap_mode="r")
labels_1 = np.load("burgers_50k-1_labels.npy", mmap_mode="r")
data_2 = np.load("burgers_50k-2.npy", mmap_mode="r")
labels_2 = np.load("burgers_50k-2_labels.npy", mmap_mode="r")
data_3 = np.load("burgers_50k-3.npy", mmap_mode="r")
labels_3 = np.load("burgers_50k-3_labels.npy", mmap_mode="r")
data_4 = np.load("burgers_50k-4.npy", mmap_mode="r")
labels_4 = np.load("burgers_50k-4_labels.npy", mmap_mode="r")
data_5 = np.load("burgers_50k-5.npy", mmap_mode="r")
labels_5 = np.load("burgers_50k-5_labels.npy", mmap_mode="r")
print(data_1.shape)
print(labels_1.shape)

(10000, 400)
(10000,)


In [4]:
xs = np.linspace(0, 1, num=400)
all_data = np.vstack((data_1, data_2, data_3, data_4, data_5))
all_labels = np.hstack((labels_1, labels_2, labels_3, labels_4, labels_5))

In [5]:
def show(index):
    plt.plot(xs, all_data[index])
    print(all_labels[index])

In [6]:
final_size = int((((400-20)/4)-15)/4) * 8
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 8, 21)
        self.pool = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(8, 8, 16)
        self.fc1 = nn.Linear(final_size, 80)
        self.fc2 = nn.Linear(80, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, final_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
def run_one_batch():
    index_subset = np.random.choice(50000, 2000, replace=False)
    data_subset = all_data[index_subset, :]
    label_subset = all_labels[index_subset]
    
    train_data = torch.from_numpy(np.expand_dims(data_subset[100:], 1))
    test_data = torch.from_numpy(np.expand_dims(data_subset[:100], 1))
    train_labels = torch.from_numpy(label_subset[100:].astype("int64"))
    test_labels = torch.from_numpy(label_subset[:100].astype("int64"))
    
    
    optimizer.zero_grad()

    outputs = net(train_data)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    
    outputs = net(test_data)
    loss2 = criterion(outputs, test_labels)
    return loss.item(), loss2.item()


In [10]:
# running_loss1 = 0.0
# running_loss2 = 0.0
# for i in range(1000):
#     loss1, loss2 = run_one_batch()
#     running_loss1 += loss1
#     running_loss2 += loss2
#     if i % 100 == 99:
#         print("Ran {} trials, training: {}, testing: {}".format(i+1, running_loss1/100, running_loss2/100))
#         running_loss1 = 0.0
#         running_loss2 = 0.0

In [11]:
# torch.save(net.state_dict(), "burgers_net_gen.pth")


In [16]:
model = Net()
model.load_state_dict(torch.load("burgers_net_gen.pth"))
model.eval()

Net(
  (conv1): Conv1d(1, 8, kernel_size=(21,), stride=(1,))
  (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 8, kernel_size=(16,), stride=(1,))
  (fc1): Linear(in_features=160, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=2, bias=True)
)

In [17]:
data_unseen = np.load("burgers_testing_only.npy", mmap_mode="r")
labels_unseen = np.load("burgers_testing_only_labels.npy", mmap_mode="r")
unseen_data = torch.from_numpy(np.expand_dims(data_unseen, 1))
unseen_labels = torch.from_numpy(labels_unseen.astype("int64"))
unseen_outputs = model(unseen_data)
unseen_loss = criterion(unseen_outputs, unseen_labels)
unseen_predictions = unseen_outputs[:, 1] > unseen_outputs[:, 0]
correct = (unseen_labels == 0) ^ unseen_predictions
total_count = len(correct)
hits = int(correct.sum())
print("Predictions: ", len(correct))
print("Hits:        ", hits)
print("Accuracy:    ", hits / total_count)
print("Totally new data loss: ", unseen_loss.item())

Predictions:  10000
Hits:         9661
Accuracy:     0.9661
Totally new data loss:  0.09024685621261597


In [18]:
data_unseen = np.load("burgers_alt.npy", mmap_mode="r")
labels_unseen = np.load("burgers_alt_labels.npy", mmap_mode="r")
unseen_data = torch.from_numpy(np.expand_dims(data_unseen, 1))
unseen_labels = torch.from_numpy(labels_unseen.astype("int64"))
unseen_outputs = model(unseen_data)
unseen_loss = criterion(unseen_outputs, unseen_labels)
unseen_predictions = unseen_outputs[:, 1] > unseen_outputs[:, 0]
correct = (unseen_labels == 0) ^ unseen_predictions
total_count = len(correct)
hits = int(correct.sum())
print("Predictions: ", len(correct))
print("Hits:        ", hits)
print("Accuracy:    ", hits / total_count)
print("Totally new data loss: ", unseen_loss.item())

Predictions:  10000
Hits:         9461
Accuracy:     0.9461
Totally new data loss:  0.154155895113945
